# 체크포인터를 사용한 상태관리

In [1]:
# 랭그래프의 체크포인터 시스템은 상태 영속성과 오류 복구를 위한 핵심 기능입니다. 
# 상태 영속성이란 그래프 실행 중 각 노드의 상태를 저장한다는 의미입니다. 
# 여러 대화나 세션의 상태를 독립적으로 관리할 수 있어서 동시에 여러 워크플로를 처리할 수 있습니다. 
# 또한 시스템 장애 시에도 마지막 체크포인트부터 재시작을 할 수 있습니다. 
# 이전에는 대화의 이력을 저장하려면 별도로 코드 작업을 해야 했는데, 
# 랭그래프에서는 이런 작업을 설정 한 줄로 끝낼 수 있도록 지원하고 있습니다.

In [ ]:
# 사용 방법
#  1. 체크포인터 설정
#  2. 그래프에 체크포인터 연결
#  3. 스레드ID로 상태 관리

#  나머지는 랭그래프에서 알아서 동작시켜줌.

In [2]:
# 예시
"""
from langgraph.checkpoint.sqlite import SqliteSaver
# pip install langgraph-checkpoint-sqlite 필요

from langgraph.graph import StateGraph

# 1. 체크포인터 설정
checkpointer = SqliteSaver.from_conn_string(":memory:")

# 2. 그래프에 체크포인터 연결
app = StateGraph(state_schema).compile(checkpointer = checkpointer)

# 3. 스레드 ID로 상태 관리
config = {"configurable": {"thread_id": "thread-1"}}
result = app.invoke(input_date, config=config)
"""
None

In [ ]:
# 기본적으로 제공하는 체크포인터는 다음과 같습니다.

# • BaseCheckpointSaver : 추상 기본 클래스
# • InMemorySaver : 메모리 기반 구현
# • SQLiteSaver, PostgresSaver : 영구 저장소 구현

# import 

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# InMemorySaver : 랭그래프의 체크포인트 시스템 중 가장 기본적인 구현체입니다. 메모리에 상태를 저장하는 방식으로, 
#   다음과 같은 특징을 가집니다.

# • 휘발성 : 프로그램이 종료되면 데이터가 사라집니다.
# • 빠른 속도: 메모리 접근이므로 매우 빠릅니다.
# • 개발/테스트용 : 주로 프로토타이핑과 테스트에 사용됩니다.

# 프로덕션에서는 SQLiteSaver, PostgresSaver 등으로 교체하여 영구 저장할 수 있습니다.

In [4]:
from typing import Dict, Any
from langgraph.graph import StateGraph, START, END

# InMemorySaver 임포트
from langgraph.checkpoint.memory import InMemorySaver

from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
import json


# 상태 정의

In [6]:
class MemoryBotState(BaseModel):
    user_message: str = Field(default="", description="사용자 입력 메세지")
    user_name: str = Field(default="", description="사용자 이름")
    user_preference: Dict[str, Any] = Field(
        default_factory=dict, description="사용자 선호도"
    )
    response: str = Field(default="", description="최종 응답")

# 메시지 처리 노드

In [ ]:
# 사용자의 메세지를 분석아혀 상태를 업데이트 하는 역할.

# ★ InMemorySaver가 노드 실행 전에 자동으로 이전 상태를 로드하며, 노드 실행 후에 자동으로 새 상태를 저장합니다. 
# 그러므로 개발자가 별도로 대화 이력관리를 하지 않아도 됩니다.



In [7]:
llm = ChatOpenAI(model='gpt-4o')

In [13]:
def process_message(state: MemoryBotState) -> Dict[str, Any]:
    message = state.user_message
    user_name = state.user_name
    preferences = state.user_preference.copy()   # 사본!

    # 시스템 프롬프트
    system_prompt = f"""
당신은 사용자의 정보를 기억하는 메모리 봇입니다.
현재 기억하고 있는 정보:
- 사용자 이름: {user_name if user_name else "모름"}
- 좋아하는 것: {preferences.get("likes", [])}
- 싫어하는 것: {preferences.get("dislikes", [])}

사용자 메시지를 분석하여 다음 JSON 형태로 응답하세요:
{{
  "response": "사용자에게 줄 응답 메시지",
  "new_name": "새로 알게 된 이름 (없으면 null)",
  "new_likes": ["새로 알게 된 좋아하는 것들"],
  "new_dislikes": ["새로 알게 된 싫어하는 것들"]
}}    
    """

    messages = [SystemMessage(content=system_prompt), HumanMessage(content=message)]
    response = llm.invoke(messages)
    print('✅ 응답확인:', response.content)
    result = json.loads(response.content)

    # 새로운 정보 업데이트
    if result.get("new_name"):
        user_name = result['new_name']

    if result.get('new_likes'):
        preferences.setdefaults("likes", []).extend(result['new_likes'])

    if result.get("new_dislikes"):
        preferences.setdefault("dislikes", []).extend(result["new_dislikes"])

    bot_response = result.get("response", "죄송해요, 이해하지 못했어요")        


    return {
        "response": bot_response,
        "user_name": user_name,
        "user_preferences": preferences,  # 사본을 세팅함.
    }
    

## .copy() 사본생성

In [8]:
a = {'name': 'John', 'age': 40}
b = a

print(f'a={a}, b={b}')

a={'name': 'John', 'age': 40}, b={'name': 'John', 'age': 40}


In [9]:
a['name'] = 'Susan'

In [10]:
print(f'a={a}, b={b}')

a={'name': 'Susan', 'age': 40}, b={'name': 'Susan', 'age': 40}


In [11]:
b = a.copy()  # dict 의 사본객체 생성
print(f'a={a}, b={b}')

a={'name': 'Susan', 'age': 40}, b={'name': 'Susan', 'age': 40}


In [12]:
a['name'] = "홍길동"
print(f'a={a}, b={b}')

a={'name': '홍길동', 'age': 40}, b={'name': 'Susan', 'age': 40}
